In [187]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
import pymc3 as pm
import arviz as az

Data description:
combined_df= All data that present abundance + Total biomass in COPEPOD the Global Plankton Databased Project with Zooplankton BIOMASS and Zooplankton Abondance.
Output Data: pivot_df= Abundance matrix for sample who has total biomass measurment (all in ind.m^-3)
result_df= Total Carbon mass matrix for the same samples as pivot_df (all in mg C m^-3)

In [205]:
####_______ Téléchargement des données présent sur COPEPOD the Global Plankton Databased Project fournissant Zooplankton BIOMASS and Zooplankton Abondance. 
####_______ Formation d'un jeu de données "combined_csv.csv" regroupant l'ensemble des données COEPOD téléchargés


Data_PATH = '/net/meso/work/svivier/Git/Data/' #Path for output (and input for other Scripts) Data
Plot_PATH = '/net/meso/work/svivier/Git/Plots/' #Path for Output Plots
Copepod_PATH = '/net/meso/work/svivier/COPEPOD_data/Data_copepod/copepod__*/data_src/short-format/*.csv' #COPEPOD Data (the Global Plankton Databased Project)

# Liste tous les fichiers .csv dans les sous-répertoires /data_src/short-format/ des dossiers commençant par "copepod__"
csv_files = glob.glob(Copepod_PATH)

# Parcourt chaque fichier CSV et effectue une opération (par exemple, afficher le nom du fichier et les premières lignes)
for file_path in csv_files:
    #print(f"Ouverture de {file_path}")
    df = pd.read_csv(file_path, sep=',', on_bad_lines='skip', encoding_errors='replace', skiprows=15)

# Liste tous les fichiers .csv dans les sous-répertoires /data_src/short-format/ des dossiers commençant par "copepod__"
csv_files = glob.glob(Copepod_PATH)

dfs = []  # Liste pour stocker les DataFrames

# Parcourt chaque fichier CSV, le lit et ajoute le DataFrame à la liste dfs
for file in csv_files:
    df = pd.read_csv(file, sep=',', on_bad_lines='skip', encoding_errors='replace', skiprows=15) #Commence a lire les fichier .csv à partir de la ligne 16 (début des données)
    dfs.append(df)

# Concatène tous les DataFrames en un seul
combined_df = pd.concat(dfs, ignore_index=True)

# Écrit le DataFrame combiné dans un nouveau fichier CSV
combined_df.to_csv(Data_PATH + 'combined_csv.csv', index=False)

/tmp/ipykernel_30198/1489254080.py:15: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,21,23,25,26,27,28,31,32,33,34,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=',', on_bad_lines='skip', encoding_errors='replace', skiprows=15)
/tmp/ipykernel_30198/1489254080.py:15: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,21,23,25,26,27,28,29,31,32,33,34,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=',', on_bad_lines='skip', encoding_errors='replace', skiprows=15)
/tmp/ipykernel_30198/1489254080.py:15: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,21,23,25,26,27,28,29,31,32,33,34,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=',', on_bad_lines='skip', encoding_errors='replace', skiprows=15)
/tmp/ipykernel_30198/1489254080.py:15: DtypeWarni

In [206]:
combined_df

,#SHP-CRUISE,YEAR,MON,DAY,TIMEgmt,TIMEloc,LATITUDE,LONGITDE,UPPER_Z,LOWER_Z,...,RECORD-ID,DATASET-ID,SHIP,PROJ,INST,Orig-CRUISE-ID,Orig-STATION-ID,Taxa-Name,Taxa-Modifiers,Unnamed: 45
0,#----------,----,---,---,-------,-------,--------,--------,-------,-------,...,----------,----------,-----,-----,-----,NaN,NaN,NaN,NaN,NaN
1,06AQ-870708,1987,7,8,99.990,-99.000,81.290,31.260,200.0,500.0,...,0001_0271,ARK IV and ARK VIII,199,null,null,ARK IV-3,276.0,Aglantha digitale,-[ ]-,NaN
2,06AQ-870708,1987,7,8,99.990,-99.000,81.290,31.260,100.0,200.0,...,0001_0272,ARK IV and ARK VIII,199,null,null,ARK IV-3,276.0,Aglantha digitale,-[ ]-,NaN
3,06AQ-870708,1987,7,8,99.990,-99.000,81.290,31.260,50.0,100.0,...,0001_0273,ARK IV and ARK VIII,199,null,null,ARK IV-3,276.0,Aglantha digitale,-[ ]-,NaN
4,06AQ-870708,1987,7,8,99.990,-99.000,81.290,31.260,25.0,50.0,...,0001_0274,ARK IV and ARK VIII,199,null,null,ARK IV-3,276.0,Aglantha digitale,-[ ]-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802297,90BY-970725,1997,7,26,17.000,-99.000,69.978,31.970,50.0,50.0,...,0006_0046,Biological Atlas of the Arctic Seas 2000,6342,null,898,90BY9770,5A,Nitzschia seriata,-[ ]-,NaN
802298,90BY-970725,1997,7,26,17.000,-99.000,69.978,31.970,0.0,0.0,...,0006_0047,Biological Atlas of the Arctic Seas 2000,6342,null,898,90BY9770,5A,Pennales,-[ ]-,NaN
802299,90BY-970725,1997,7,26,17.000,-99.000,69.978,31.970,50.0,50.0,...,0006_0048,Biological Atlas of the Arctic Seas 2000,6342,null,898,90BY9770,5A,Pennales,-[ ]-,NaN
802300,90BY-970725,1997,7,26,17.000,-99.000,69.978,31.970,50.0,50.0,...,0006_0066,Biological Atlas of the Arctic Seas 2000,6342,null,898,90BY9770,5A,Thalassiosira spp.,-[ ]-,NaN


In [191]:
print(combined_df.columns.tolist())

['#SHP-CRUISE', 'YEAR', 'MON', 'DAY', 'TIMEgmt', 'TIMEloc', 'LATITUDE', 'LONGITDE', 'UPPER_Z', 'LOWER_Z', 'T', 'GEAR', 'MESH', 'NMFS_PGC', 'ITIS_TSN', 'MOD', 'LIF', 'PSC', 'SEX', 'V', ' Water Strained', ' Original-VALUE', 'Orig-UNITS', ' VALUE-per-volu', 'UNITS', 'F1', 'F2', 'F3', 'F4', ' VALUE-per-area', 'UNITS.1', 'F1.1', 'F2.1', 'F3.1', 'F4.1', 'SCIENTIFIC NAME -[ modifiers ]-', ' RECORD-ID', 'DATASET-ID', ' SHIP', ' PROJ', ' INST', 'Orig-CRUISE-ID', 'Orig-STATION-ID', 'Taxa-Name', 'Taxa-Modifiers', 'Unnamed: 45']


In [192]:
# We want to keep samples that has taxonomic abundance and total biomass observation, we create a unique_id (same unique_id = same samples)

In [193]:
import pandas as pd


# Liste des colonnes utilisées pour créer l'identifiant unique
columns_for_id = ['#SHP-CRUISE', 'YEAR', 'MON', 'DAY', 'TIMEgmt', 'TIMEloc', 'LATITUDE', 'LONGITDE', 'UPPER_Z', 'LOWER_Z', 'T', 'GEAR', 'MESH', ' Water Strained']

# Créer une nouvelle colonne 'unique_id' en concaténant les valeurs des colonnes spécifiées
combined_df['unique_id'] = combined_df[columns_for_id].astype(str).agg('_'.join, axis=1)

# Afficher les premières lignes pour vérifier
combined_df


,#SHP-CRUISE,YEAR,MON,DAY,TIMEgmt,TIMEloc,LATITUDE,LONGITDE,UPPER_Z,LOWER_Z,...,DATASET-ID,SHIP,PROJ,INST,Orig-CRUISE-ID,Orig-STATION-ID,Taxa-Name,Taxa-Modifiers,Unnamed: 45,unique_id
0,#----------,----,---,---,-------,-------,--------,--------,-------,-------,...,----------,-----,-----,-----,NaN,NaN,NaN,NaN,NaN,#----------_----_---_---_-------_-------_-----...
1,06AQ-870708,1987,7,8,99.990,-99.000,81.290,31.260,200.0,500.0,...,ARK IV and ARK VIII,199,null,null,ARK IV-3,276.0,Aglantha digitale,-[ ]-,NaN,06AQ-870708_1987_ 7_ 8_ 99.990_-99.000_ 81....
2,06AQ-870708,1987,7,8,99.990,-99.000,81.290,31.260,100.0,200.0,...,ARK IV and ARK VIII,199,null,null,ARK IV-3,276.0,Aglantha digitale,-[ ]-,NaN,06AQ-870708_1987_ 7_ 8_ 99.990_-99.000_ 81....
3,06AQ-870708,1987,7,8,99.990,-99.000,81.290,31.260,50.0,100.0,...,ARK IV and ARK VIII,199,null,null,ARK IV-3,276.0,Aglantha digitale,-[ ]-,NaN,06AQ-870708_1987_ 7_ 8_ 99.990_-99.000_ 81....
4,06AQ-870708,1987,7,8,99.990,-99.000,81.290,31.260,25.0,50.0,...,ARK IV and ARK VIII,199,null,null,ARK IV-3,276.0,Aglantha digitale,-[ ]-,NaN,06AQ-870708_1987_ 7_ 8_ 99.990_-99.000_ 81....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802297,90BY-970725,1997,7,26,17.000,-99.000,69.978,31.970,50.0,50.0,...,Biological Atlas of the Arctic Seas 2000,6342,null,898,90BY9770,5A,Nitzschia seriata,-[ ]-,NaN,90BY-970725_1997_ 7_ 26_ 17.000_-99.000_ 69....
802298,90BY-970725,1997,7,26,17.000,-99.000,69.978,31.970,0.0,0.0,...,Biological Atlas of the Arctic Seas 2000,6342,null,898,90BY9770,5A,Pennales,-[ ]-,NaN,90BY-970725_1997_ 7_ 26_ 17.000_-99.000_ 69....
802299,90BY-970725,1997,7,26,17.000,-99.000,69.978,31.970,50.0,50.0,...,Biological Atlas of the Arctic Seas 2000,6342,null,898,90BY9770,5A,Pennales,-[ ]-,NaN,90BY-970725_1997_ 7_ 26_ 17.000_-99.000_ 69....
802300,90BY-970725,1997,7,26,17.000,-99.000,69.978,31.970,50.0,50.0,...,Biological Atlas of the Arctic Seas 2000,6342,null,898,90BY9770,5A,Thalassiosira spp.,-[ ]-,NaN,90BY-970725_1997_ 7_ 26_ 17.000_-99.000_ 69....


In [194]:
# we create filtered_df: we keep the unique_id that has both 'V'=c (abundance) and 'V'=b (biomass)

import pandas as pd


# Liste des colonnes utilisées pour créer l'identifiant unique: Same unique_id = Same sample
columns_for_id = ['#SHP-CRUISE', 'YEAR', 'MON', 'DAY', 'TIMEgmt', 'TIMEloc', 'LATITUDE', 'LONGITDE', 'UPPER_Z', 'LOWER_Z', 'T', 'GEAR', 'MESH', ' Water Strained']

# Créer une nouvelle colonne 'unique_id' en concaténant les valeurs des colonnes spécifiées
combined_df['unique_id'] = combined_df[columns_for_id].astype(str).agg('_'.join, axis=1)

# Filtrer les unique_id qui ont des lignes avec 'V' égal à 'c' = Abundance data
unique_ids_with_c = set(combined_df[combined_df['V'] == 'c']['unique_id'])

# Filtrer les unique_id qui ont des lignes avec 'V' égal à 'b' = Totalbiomass data
unique_ids_with_b = set(combined_df[combined_df['V'] == 'b']['unique_id'])

# Trouver les unique_id qui ont à la fois 'V' égal à 'c' et 'V' égal à 'b': Samples with both abundance data and Totalbiomass data
unique_ids_with_c_and_b = unique_ids_with_c.intersection(unique_ids_with_b)

# Filtrer le DataFrame pour ne conserver que les lignes avec les unique_id identifiés
filtered_df = combined_df[combined_df['unique_id'].isin(unique_ids_with_c_and_b)]

# Afficher les premières lignes pour vérifier
filtered_df



,#SHP-CRUISE,YEAR,MON,DAY,TIMEgmt,TIMEloc,LATITUDE,LONGITDE,UPPER_Z,LOWER_Z,...,DATASET-ID,SHIP,PROJ,INST,Orig-CRUISE-ID,Orig-STATION-ID,Taxa-Name,Taxa-Modifiers,Unnamed: 45,unique_id
5623,06AQ-910604,1991,6,8,99.990,-99.000,79.017,-3.233,0.0,100.0,...,ARK IV and ARK VIII,199,435,892,ARK VIII-1,5.0,Zooplankton dry weight,-[ ]-,NaN,06AQ-910604_1991_ 6_ 8_ 99.990_-99.000_ 79....
5624,06AQ-910604,1991,6,8,99.990,-99.000,79.017,-3.233,0.0,100.0,...,ARK IV and ARK VIII,199,435,892,ARK VIII-1,5.0,Zooplankton carbon mass,-[ ]-,NaN,06AQ-910604_1991_ 6_ 8_ 99.990_-99.000_ 79....
5625,06AQ-910604,1991,6,8,99.990,-99.000,79.017,-3.233,0.0,100.0,...,ARK IV and ARK VIII,199,435,892,ARK VIII-1,5.0,Calanus spp.,-[ c1 ]-,NaN,06AQ-910604_1991_ 6_ 8_ 99.990_-99.000_ 79....
5626,06AQ-910604,1991,6,8,99.990,-99.000,79.017,-3.233,0.0,100.0,...,ARK IV and ARK VIII,199,435,892,ARK VIII-1,5.0,Calanus spp.,-[ c2 ]-,NaN,06AQ-910604_1991_ 6_ 8_ 99.990_-99.000_ 79....
5627,06AQ-910604,1991,6,8,99.990,-99.000,79.017,-3.233,0.0,100.0,...,ARK IV and ARK VIII,199,435,892,ARK VIII-1,5.0,Calanus finmarchicus,-[ c3-6 ]-,NaN,06AQ-910604_1991_ 6_ 8_ 99.990_-99.000_ 79....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801975,90DI-521028,1952,11,2,10.420,-99.000,68.342,38.750,10.0,25.0,...,Biological Atlas of the Arctic Seas 2000,7552,null,898,90DI52A0,41B,Verruca spp.,-[ ]-,NaN,90DI-521028_1952_ 11_ 2_ 10.420_-99.000_ 68....
801976,90DI-521028,1952,11,2,10.420,-99.000,68.342,38.750,0.0,10.0,...,Biological Atlas of the Arctic Seas 2000,7552,null,898,90DI52A0,41B,Verruca spp.,-[ ]-,NaN,90DI-521028_1952_ 11_ 2_ 10.420_-99.000_ 68....
801979,90DI-521028,1952,11,2,10.420,-99.000,68.342,38.750,25.0,50.0,...,Biological Atlas of the Arctic Seas 2000,7552,null,898,90DI52A0,41B,Limacina spp.,-[ ]-,NaN,90DI-521028_1952_ 11_ 2_ 10.420_-99.000_ 68....
801980,90DI-521028,1952,11,2,10.420,-99.000,68.342,38.750,10.0,25.0,...,Biological Atlas of the Arctic Seas 2000,7552,null,898,90DI52A0,41B,Limacina spp.,-[ ]-,NaN,90DI-521028_1952_ 11_ 2_ 10.420_-99.000_ 68....


# Convertion in ind.m^-3 for abundance data and convertion in mg.C.m^-3 for Biomass data

In [195]:
# Séparer les jeux de données
df_with_c = filtered_df[filtered_df['V'] == 'c']
df_with_b = filtered_df[filtered_df['V'] == 'b']

# Afficher les valeurs distinctes de 'Orig-UNITS' pour chaque jeu de données
orig_units_with_c = df_with_c['UNITS'].value_counts()
orig_units_with_b = df_with_b['UNITS'].value_counts()

print("Valeurs distinctes de 'UNITS' pour les lignes avec 'V' = 'c':")
print(orig_units_with_c)

print("\nValeurs distinctes de 'Orig-UNITS' pour les lignes avec 'V' = 'b':")
orig_units_with_b

Valeurs distinctes de 'UNITS' pour les lignes avec 'V' = 'c':
UNITS
 #/m3    298277
 #/ml     11897
-----      2665
 #/mL        23
Name: count, dtype: int64

Valeurs distinctes de 'Orig-UNITS' pour les lignes avec 'V' = 'b':


UNITS
ml/m3    4927
mg/m3    4014
-----    3802
Name: count, dtype: int64

In [196]:
# Convertion in ind.m^-3 for abundance data and convertion in mg.C.m^-3 for Biomass data

df_with_c['LOWER_Z'] = pd.to_numeric(df_with_c['LOWER_Z'], errors='coerce')
df_with_c['UPPER_Z'] = pd.to_numeric(df_with_c['UPPER_Z'], errors='coerce')
df_with_c[' VALUE-per-volu'] = pd.to_numeric(df_with_c[' VALUE-per-volu'], errors='coerce')
df_with_c[' Water Strained'] = pd.to_numeric(df_with_c[' Water Strained'], errors='coerce')

df_with_b['LOWER_Z'] = pd.to_numeric(df_with_b['LOWER_Z'], errors='coerce')
df_with_b['UPPER_Z'] = pd.to_numeric(df_with_b['UPPER_Z'], errors='coerce')
df_with_b[' VALUE-per-volu'] = pd.to_numeric(df_with_b[' VALUE-per-volu'], errors='coerce')
df_with_b[' Water Strained'] = pd.to_numeric(df_with_b[' Water Strained'], errors='coerce')

# Fonction pour calculer 'MeasurementValue_m^3' pour V=c
def calculate_measurement_value_c(row):
    if row['UNITS'].strip() == '#/m3':
        return row[' VALUE-per-volu']
    elif row['UNITS'].strip() == '#/ml':
        return row[' VALUE-per-volu'] * 1000000
    elif row['UNITS'].strip() == '#/mL':
        return row[' VALUE-per-volu'] * 1000000
    else:
        return None

# Appliquer la fonction pour V=c
df_with_c['MeasurementValue_m^3'] = df_with_c.apply(calculate_measurement_value_c, axis=1)

# Fonction pour calculer 'MeasurementValue_m^3' pour V=b
def calculate_measurement_value_b(row):
    if row['UNITS'].strip() == 'ml/m3':
        return row[' VALUE-per-volu']
    elif row['UNITS'].strip() == 'mg/m3':
        return row[' VALUE-per-volu']
    elif row['UNITS'].strip() == '-----':
        return row[' VALUE-per-volu']
    else:
        return None

# Appliquer la fonction pour V=b
df_with_b['MeasurementValue_m^3'] = df_with_b.apply(calculate_measurement_value_b, axis=1)

# Afficher les premières lignes pour vérifier
print("Pour V=c:")
print(df_with_c[['Orig-UNITS', ' Original-VALUE', 'MeasurementValue_m^3']].head())

print("\nPour V=b:")
print(df_with_b[['Orig-UNITS', ' Original-VALUE', 'MeasurementValue_m^3']].head())

/tmp/ipykernel_30198/1461890208.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_c['LOWER_Z'] = pd.to_numeric(df_with_c['LOWER_Z'], errors='coerce')
/tmp/ipykernel_30198/1461890208.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_c['UPPER_Z'] = pd.to_numeric(df_with_c['UPPER_Z'], errors='coerce')
/tmp/ipykernel_30198/1461890208.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

Pour V=c:
      Orig-UNITS   Original-VALUE  MeasurementValue_m^3
5625        #/m3              1.6                   1.6
5626        #/m3               0.                   0.0
5627        #/m3              5.7                   5.7
5628        #/m3              4.4                   4.4
5629        #/m3              4.3                   4.3

Pour V=b:
      Orig-UNITS   Original-VALUE  MeasurementValue_m^3
5623       mg/m2             800.                   8.0
5624       kg/m2             320.                   3.2
5632       mg/m2            3100.                  31.0
5633       kg/m2            1240.                  12.4
5709       mg/m2             700.                   7.0


/tmp/ipykernel_30198/1461890208.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_c['MeasurementValue_m^3'] = df_with_c.apply(calculate_measurement_value_c, axis=1)
/tmp/ipykernel_30198/1461890208.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_b['MeasurementValue_m^3'] = df_with_b.apply(calculate_measurement_value_b, axis=1)


In [197]:
# Enregistrer V=b dans un fichier CSV nommé 'Biomass.csv'
df_with_b.to_csv(Data_PATH + 'Biomass.csv', index=False)

# Enregistrer V=c dans un fichier CSV nommé 'Abundance.csv'
df_with_c.to_csv(Data_PATH + 'Abundance.csv', index=False)


## Biomass into CarbonMass

In [198]:
#The different biomass measurement in COPEPOD:
print(df_with_b['Taxa-Name'].unique())

['Zooplankton dry weight' 'Zooplankton carbon mass'
 'Zooplankton settled volume' 'Zooplankton wet weight'
 'Zooplankton displacement volume' 'Zooplankton ashfree dry weight']


In [199]:
# Différentes mesure de biomasse => Convertion en carbon mass totale

In [200]:
# We use well known conversion equation in the litterature

import numpy as np
import pandas as pd

def convert_to_carbon_mass(row):
    value = row['MeasurementValue_m^3']
    taxa_name = row['Taxa-Name']
    
    if taxa_name == 'Zooplankton displacement volume':
        # Displacement Volume (DV) -> Carbon Mass (CM)
        log_CM = (np.log10(value) + 1.434) / 0.820 #Wiebe (1988)
        carbon_mass = 10 ** log_CM
    elif taxa_name == 'Zooplankton wet weight':
        # Wet Mass (WM) -> Dry Mass (DM) #Weibe (1985)
        log_DM = 0.18 * value
        dry_mass = log_DM
        # Dry Mass (DM) -> Carbon Mass (CM) #Weibe (1985)
        log_CM = (np.log10(dry_mass) - 0.499) / 0.991
        carbon_mass = 10 ** log_CM
    elif taxa_name == 'Zooplankton dry weight':
        # Dry Mass (DM) -> Carbon Mass (CM)
        log_CM = (np.log10(value) - 0.499) / 0.991 #Weibe (1985)
        carbon_mass = 10 ** log_CM
    elif taxa_name == 'Zooplankton settled volume':
        # Settled Volume (SV) -> Dry Mass (DM)
        log_DM = 0.843 * np.log10(value) + 1.417 #Balvay (1987)
        dry_mass = 10 ** log_DM
        # Dry Mass (DM) -> Carbon Mass (CM)
        log_CM = (np.log10(dry_mass) - 0.499) / 0.991 #Weibe (1985)
        carbon_mass = 10 ** log_CM
    elif taxa_name == 'Zooplankton ashfree dry weight':
        # Ash-free Dry Mass (AFDM) -> Carbon Mass (CM)
        log_CM = (np.log10(value) - 0.410) / 0.963 #Bode et al. (1998)
        carbon_mass = 10 ** log_CM
    elif taxa_name == 'Zooplankton carbon mass':
        # Si c'est déjà de la masse de carbone, on garde la valeur telle quelle
        carbon_mass = value
    else:
        # Si le nom du taxa n'est pas reconnu, retourner NaN
        carbon_mass = np.nan
    
    return carbon_mass



# Appliquer la conversion à chaque ligne du DataFrame
df_with_b['Carbon Mass'] = df_with_b.apply(convert_to_carbon_mass, axis=1)

# Afficher les premières lignes pour vérifier les résultats
print(df_with_b[['Taxa-Name', 'MeasurementValue_m^3', 'Carbon Mass']].head(10))

# Enregistrer le DataFrame mis à jour dans un fichier CSV nommé 'Biomass.csv'
df_with_b.to_csv(Data_PATH + 'Biomass.csv', index=False)


/tmp/ipykernel_30198/1618446962.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_b['Carbon Mass'] = df_with_b.apply(convert_to_carbon_mass, axis=1)


                    Taxa-Name  MeasurementValue_m^3  Carbon Mass
5623   Zooplankton dry weight                   8.0     2.557171
5624  Zooplankton carbon mass                   3.2     3.200000
5632   Zooplankton dry weight                  31.0    10.031689
5633  Zooplankton carbon mass                  12.4    12.400000
5709   Zooplankton dry weight                   7.0     2.234813
5710  Zooplankton carbon mass                   2.8     2.800000
5716   Zooplankton dry weight                  49.0    15.922608
5717  Zooplankton carbon mass                  19.6    19.600000
5725   Zooplankton dry weight                  23.0     7.422717
5726  Zooplankton carbon mass                   9.2     9.200000


# Abundance Matrix and Biomass Matrix

In [201]:
import pandas as pd

# Grouper par 'unique_id' et 'Taxa-Name' et sommer les 'MeasurementValue_m^3'
grouped_df = df_with_c.groupby(['unique_id', 'Taxa-Name'])['MeasurementValue_m^3'].sum().reset_index()

# Créer une matrice pivot
pivot_df = grouped_df.pivot(index='unique_id', columns='Taxa-Name', values='MeasurementValue_m^3')

# Remplacer les NaN par 0 si nécessaire
pivot_df = pivot_df.fillna(0)

# Afficher les premières lignes pour vérifier les résultats
print(pivot_df.head())

# Enregistrer la matrice dans un fichier CSV si nécessaire
pivot_df.to_csv(Data_PATH + 'MeasurementValue_matrix.csv')


Taxa-Name                                           Acartia  Acartia bifilosa  \
unique_id                                                                       
06AQ-910604_1991_  6_  8_ 99.990_-99.000_  79.0...      0.0               0.0   
06AQ-910604_1991_  6_  9_ 99.990_-99.000_  78.6...      0.0               0.0   
06AQ-910604_1991_  6_  9_ 99.990_-99.000_  78.9...      0.0               0.0   
06AQ-910604_1991_  6_  9_ 99.990_-99.000_  79.0...      0.0               0.0   
06AQ-910604_1991_  6_ 10_ 99.990_-99.000_  78.8...      0.0               0.0   

Taxa-Name                                           Acartia longiremis  \
unique_id                                                                
06AQ-910604_1991_  6_  8_ 99.990_-99.000_  79.0...                 0.0   
06AQ-910604_1991_  6_  9_ 99.990_-99.000_  78.6...                 0.0   
06AQ-910604_1991_  6_  9_ 99.990_-99.000_  78.9...                 0.0   
06AQ-910604_1991_  6_  9_ 99.990_-99.000_  79.0...            

In [202]:
import pandas as pd


# Grouper par 'unique_id' et calculer la moyenne des 'Carbon Mass'
grouped_df = df_with_b.groupby('unique_id')['Carbon Mass'].mean().reset_index()

# Créer un DataFrame avec 'unique_id' comme index
result_df = grouped_df.set_index('unique_id')

# Afficher les premières lignes pour vérifier les résultats
print(result_df.head())

# Enregistrer le DataFrame dans un fichier CSV si nécessaire
result_df.to_csv(Data_PATH + 'CarbonMass_matrix.csv')


                                                    Carbon Mass
unique_id                                                      
06AQ-910604_1991_  6_  8_ 99.990_-99.000_  79.0...     2.878586
06AQ-910604_1991_  6_  9_ 99.990_-99.000_  78.6...    17.761304
06AQ-910604_1991_  6_  9_ 99.990_-99.000_  78.9...     2.517407
06AQ-910604_1991_  6_  9_ 99.990_-99.000_  79.0...    11.215845
06AQ-910604_1991_  6_ 10_ 99.990_-99.000_  78.8...     8.311358


In [203]:
result_df

,Carbon Mass
unique_id,
06AQ-910604_1991_ 6_ 8_ 99.990_-99.000_ 79.017_ -3.233_ 0.0_ 100.0_V_ 118_ 200_ 30.0 m3,2.878586
06AQ-910604_1991_ 6_ 9_ 99.990_-99.000_ 78.667_ -7.550_ 0.0_ 100.0_V_ 118_ 200_ 30.0 m3,17.761304
06AQ-910604_1991_ 6_ 9_ 99.990_-99.000_ 78.950_ -6.750_ 0.0_ 100.0_V_ 118_ 200_ 30.0 m3,2.517407
06AQ-910604_1991_ 6_ 9_ 99.990_-99.000_ 79.017_ -4.883_ 0.0_ 100.0_V_ 118_ 200_ 30.0 m3,11.215845
06AQ-910604_1991_ 6_ 10_ 99.990_-99.000_ 78.817_ -9.217_ 0.0_ 100.0_V_ 118_ 200_ 30.0 m3,8.311358
...,...
T028-990522_1999_ 6_ 4_ 20.317_ 20.317_ 44.876_ -1.580_ 0.0_ 71.0_V_ 285_ 150_ 3.6 m3,3.369213
T028-990522_1999_ 6_ 4_ 20.750_ 20.750_ 44.876_ -1.508_ 0.0_ 54.0_V_ 285_ 150_ 2.7 m3,4.284212
T028-990522_1999_ 6_ 4_ 21.217_ 21.217_ 44.876_ -1.436_ 0.0_ 47.0_V_ 285_ 150_ 2.4 m3,1.331615


In [204]:
pivot_df

Taxa-Name,Acartia,Acartia bifilosa,Acartia longiremis,Acartia spp.,Aetideopsis multiserrata,Aglantha digitale,Aglantha spp.,Agonidae,Aidanosagitta crassa naikaiensis,Aidanosagitta regularis,...,Zooplankton sp. H,Zooplankton sp. I,Zooplankton sp. J,Zooplankton sp. K,Zooplankton sp. L,Zooplankton sp. M,Zooplankton unidentified,Zooplankton various,fish (unidentified eggs or larvae),fish (unidentified eggs or larvae) TOTAL (total count)
unique_id,,,,,,,,,,,,,,,,,,,,,
06AQ-910604_1991_ 6_ 8_ 99.990_-99.000_ 79.017_ -3.233_ 0.0_ 100.0_V_ 118_ 200_ 30.0 m3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06AQ-910604_1991_ 6_ 9_ 99.990_-99.000_ 78.667_ -7.550_ 0.0_ 100.0_V_ 118_ 200_ 30.0 m3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06AQ-910604_1991_ 6_ 9_ 99.990_-99.000_ 78.950_ -6.750_ 0.0_ 100.0_V_ 118_ 200_ 30.0 m3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06AQ-910604_1991_ 6_ 9_ 99.990_-99.000_ 79.017_ -4.883_ 0.0_ 100.0_V_ 118_ 200_ 30.0 m3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06AQ-910604_1991_ 6_ 10_ 99.990_-99.000_ 78.817_ -9.217_ 0.0_ 100.0_V_ 118_ 200_ 30.0 m3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T028-990522_1999_ 6_ 4_ 20.317_ 20.317_ 44.876_ -1.580_ 0.0_ 71.0_V_ 285_ 150_ 3.6 m3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T028-990522_1999_ 6_ 4_ 20.750_ 20.750_ 44.876_ -1.508_ 0.0_ 54.0_V_ 285_ 150_ 2.7 m3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T028-990522_1999_ 6_ 4_ 21.217_ 21.217_ 44.876_ -1.436_ 0.0_ 47.0_V_ 285_ 150_ 2.4 m3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
